In [ ]:
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
train_data, val_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

In [ ]:
train_example_batch,train_labels_batch=next(iter(train_data.batch(5)))

In [ ]:
train_example_batch

<tf.Tensor: shape=(5,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell a

In [ ]:
train_labels_batch

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([0, 0, 0, 1, 1])>

In [ ]:
hub_layer = hub.KerasLayer("https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1", output_shape=[20],
                           input_shape=[], dtype=tf.string,trainable=True)

In [ ]:
hub_layer(train_example_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_6 (Dense)             (None, 16)                336       
                                                                 
 dense_7 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
history=model.fit(train_data.shuffle(10000).batch(100),epochs=20,validation_data=val_data.batch(100),verbose=1)

Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


150/150 [==============================] - 7s 39ms/step - loss: 0.6163 - accuracy: 0.6605 - val_loss: 0.5566 - val_accuracy: 0.7277
Epoch 2/20
150/150 [==============================] - 4s 23ms/step - loss: 0.5008 - accuracy: 0.7644 - val_loss: 0.4628 - val_accuracy: 0.7940
Epoch 3/20
150/150 [==============================] - 5s 32ms/step - loss: 0.4019 - accuracy: 0.8283 - val_loss: 0.3905 - val_accuracy: 0.8319
Epoch 4/20
150/150 [==============================] - 3s 21ms/step - loss: 0.3234 - accuracy: 0.8679 - val_loss: 0.3446 - val_accuracy: 0.8525
Epoch 5/20
150/150 [==============================] - 4s 23ms/step - loss: 0.2679 - accuracy: 0.8955 - val_loss: 0.3199 - val_accuracy: 0.8665
Epoch 6/20
150/150 [==============================] - 5s 32ms/step - loss: 0.2287 - accuracy: 0.9136 - val_loss: 0.3062 - val_accuracy: 0.8724
Epoch 7/20
150/150 [==============================] - 4s 23ms/step - loss: 0.1967 - accuracy: 0.9266 - val_loss: 0.3008 - val_accuracy: 0.8740
Epoch 8/20

In [ ]:
results = model.evaluate(test_data.batch(100), verbose=2)
results


250/250 - 2s - loss: 0.5259 - accuracy: 0.8478 - 2s/epoch - 10ms/step


[0.525913655757904, 0.8478400111198425]